In [1]:
import numpy as np
import octomap
import pclpy
import matplotlib.pyplot as plt
import numpy as np
import bbox
import open3d as o3d
import random
from copy import deepcopy
import pandas as pd
import networkx as nx
import itertools
from scipy.spatial.distance import cdist,euclidean
import time
import heapq

In [2]:
cloud = pclpy.pcl.PointCloud.PointXYZRGB()
#pclpy.pcl.io.loadPCDFile('segmented_cloud.pcd',cloud)
#pclpy.pcl.io.loadPCDFile('/home/gyk/data/trees_globcen.pcd',cloud)
#pclpy.pcl.io.loadPCDFile('/home/gyk/data/big_env.pcd',cloud)
pclpy.pcl.io.loadPCDFile('/home/gyk/data/treedense1.pcd',cloud)

plist=[]
for point in cloud.points:
    plist.append([point.x,point.y,point.z])
parray= np.asarray(plist)
parray.astype(float)

#parameters
resolution= 1
origin=np.array([18, 13.5,11])

#create octree
tree = octomap.OcTree(resolution)
for every in parray: 
    tree.insertPointCloud(np.asarray([every]),np.array([every[0],every[1],origin[2]]))

occupied,empty = tree.extractPointCloud()
len(empty)

8288

In [8]:
def find_path_lenth(waypts):
    length=0
    for i in range(len(waypts)-1):
        length += euc_dis(waypts[i],waypts[i+1])
    return length

def find_neigh(every,data_arr):
    nb=[]
    for j in range(data_arr.shape[0]):
        if checkconnection(every,data_arr[j]):
            nb.append(data_arr[j])
    return nb

def checkconnection(a,b):
    a_coords=a[:3]
    b_coords=b[:3]
    a_size=1
    b_size=1
    if condition(a_coords,b_coords,a_size,b_size)== True:
        return True
    return False
 
def condition(coords_a,coords_b,size_a,size_b):
    size_cond= (size_a+size_b)/2
    count=0
    for i in range(len(coords_a)):
        if coords_a[i]==coords_b[i]:
            count +=1 
    if abs(coords_a[0]-coords_b[0]) <= size_cond:
            if abs(coords_a[1]-coords_b[1]) <= size_cond:
                    if abs(coords_a[2]-coords_b[2]) <= size_cond:
                        if count==2:
                            return True
    return False

def return_path(current_node):
    path = []
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    return path[::-1]  # Return reversed path

def save_as_csv(D_graph,file_path,mode,waypts):
    '''
    mode=1 for save data
    mode=2 for saving nodes 
    '''
    if mode==1:
        arr=[]
        for key,vals in D_graph.items():
            for subs,values in vals.items():
                if subs!='nodes':
                    for i in values:
                        arr.append(i)
        arr= np.asarray(arr)

        one_ar= np.ones(arr.shape[0])

        fine_arr= np.hstack((arr,one_ar[np.newaxis].T))

        dfempty = pd.DataFrame({'Column1': fine_arr[:, 0], 'Column2': fine_arr[:, 1],
                                'Column3': fine_arr[:, 2],'Column4': fine_arr[:, 3]})
        dfempty.to_csv(file_path)
        
        return True
    
    if mode==2:
        arr=[]
        for key,vals in D_graph.items():
            for subs,values in vals.items():
                if subs=='nodes':
                    for i in values:
                        arr.append(i)
        arr= np.asarray(arr)

        one_ar= np.ones(arr.shape[0])

        fine_arr= np.hstack((arr,one_ar[np.newaxis].T))

        dfempty = pd.DataFrame({'Column1': fine_arr[:, 0], 'Column2': fine_arr[:, 1],
                                'Column3': fine_arr[:, 2],'Column4': fine_arr[:, 3]})
        dfempty.to_csv(file_path)
        return True
    
    if mode==3:
        arr=np.asarray(waypoints)
        one_ar= np.ones(arr.shape[0])

        fine_arr= np.hstack((arr,one_ar[np.newaxis].T))

        dfempty = pd.DataFrame({'Column1': fine_arr[:, 0], 'Column2': fine_arr[:, 1],
                                'Column3': fine_arr[:, 2],'Column4': fine_arr[:, 3]})
        dfempty.to_csv(file_path)
        return True
    
    return False

def euc_dis(a,b):
    a,b= np.asarray([a]),np.asarray([b])
    return euclidean(a,b)

In [9]:
class Node:
    """
    A node class for A* Pathfinding
    """

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
    
    def __repr__(self):
        return f"{self.position} - g: {self.g} h: {self.h} f: {self.f}"

    # defining less than for purposes of heap queue
    def __lt__(self, other):
        return self.f < other.f
    
    # defining greater than for purposes of heap queue
    def __gt__(self, other):
        return self.f > other.f
    
def astar(data_arr, start, end, allow_diagonal_movement = False):
    """
    Returns a list of tuples as a path from the given start to the given end in the given maze
    :param maze:
    :param start:
    :param end:
    :return:
    """

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Heapify the open_list and Add the start node
    heapq.heapify(open_list) 
    heapq.heappush(open_list, start_node)

    # Adding a stop condition
    outer_iterations = 0
    max_iterations = 10000000

    # what squares do we search
    adjacent_squares = ((0, -1), (0, 1), (-1, 0), (1, 0),)
    if allow_diagonal_movement:
        adjacent_squares = ((0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1),)

    # Loop until you find the end
    while len(open_list) > 0:
        outer_iterations += 1

        if outer_iterations > max_iterations:
          # if we hit this point return the path such as it is
          # it will not contain the destination
          #warn("giving up on pathfinding too many iterations")
          return return_path(current_node)       
        
        # Get the current node
        current_node = heapq.heappop(open_list)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            return return_path(current_node)

        # Generate children
        children = []
        
        neighs= find_neigh(current_node.position,data_arr)
        
        for every in neighs:
        # Create new node
            every=tuple(every)
            new_node = Node(current_node, every)
            # Append
            children.append(new_node)

        # Loop through children
        for child in children:
            # Child is on the closed list
            if len([closed_child for closed_child in closed_list if closed_child == child]) > 0:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)+((child.position[2] - end_node.position[2]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            if len([open_node for open_node in open_list if child.position == open_node.position and child.g > open_node.g]) > 0:
                continue

            # Add the child to the open list
            heapq.heappush(open_list, child)

    #warn("Couldn't get a path to destination")
    return None



def save_as_csv(D_graph,file_path,mode,waypts):
    '''
    mode=1 for save data
    mode=2 for saving nodes 
    '''
    if mode==1:
        arr=[]
        for key,vals in D_graph.items():
            for subs,values in vals.items():
                if subs!='nodes':
                    for i in values:
                        arr.append(i)
        arr= np.asarray(arr)

        one_ar= np.ones(arr.shape[0])

        fine_arr= np.hstack((arr,one_ar[np.newaxis].T))

        dfempty = pd.DataFrame({'Column1': fine_arr[:, 0], 'Column2': fine_arr[:, 1],
                                'Column3': fine_arr[:, 2],'Column4': fine_arr[:, 3]})
        dfempty.to_csv(file_path)
        
        return True
    
    if mode==2:
        arr=[]
        for key,vals in D_graph.items():
            for subs,values in vals.items():
                if subs=='nodes':
                    for i in values:
                        arr.append(i)
        arr= np.asarray(arr)

        one_ar= np.ones(arr.shape[0])

        fine_arr= np.hstack((arr,one_ar[np.newaxis].T))

        dfempty = pd.DataFrame({'Column1': fine_arr[:, 0], 'Column2': fine_arr[:, 1],
                                'Column3': fine_arr[:, 2],'Column4': fine_arr[:, 3]})
        dfempty.to_csv(file_path)
        return True
    
    if mode==3:
        arr=np.asarray(waypoints)
        one_ar= np.ones(arr.shape[0])

        fine_arr= np.hstack((arr,one_ar[np.newaxis].T))

        dfempty = pd.DataFrame({'Column1': fine_arr[:, 0], 'Column2': fine_arr[:, 1],
                                'Column3': fine_arr[:, 2],'Column4': fine_arr[:, 3]})
        dfempty.to_csv(file_path)
        return True
    
    return False

def Astar_alg(start,end,print_maze = True):
    path = astar(np.asarray(empty), start, end)
    return path

In [5]:
df=pd.read_csv('/home/gyk/data/prm/evaluation.csv')
df_out= pd.DataFrame(columns=['sx','sy','sz','ex','ey','ez','time','pathlength'])

In [9]:
start_l=list(df[['sx','sy','sz']].to_numpy())
end_l=list(df[['ex','ey','ez']].to_numpy())
print(start_l)

for every in range(len(start_l)):
    stime= time.time()
    path_f=Astar_alg(tuple(start_l[every]),tuple(end_l[every]),True)
    time_f= np.asarray(time.time()-stime)
    waypoints=[]
    for i in path_f:
        waypoints.append(list(i))
    #print(waypoints)
    pathlength=np.asarray(find_path_lenth(waypoints))
    pos=(np.hstack((np.asarray(start_l[every]),np.asarray(end_l[every]),time_f,pathlength)))
    df_temp=pd.DataFrame([pos],columns=['sx','sy','sz','ex','ey','ez','time','pathlength'])
    df_out= df_out.append(df_temp,ignore_index=True)
#print(path_f)

[array([24.5,  0.5, 11.5]), array([12.5,  3.5, 10.5]), array([20.5,  7.5,  2.5]), array([19.5, 19.5,  5.5]), array([3.5, 1.5, 4.5])]


KeyboardInterrupt: 

In [12]:
# for i in empty:
#     if int(i[0])==1 and int(i[1])==1 and int(i[2])==2:
#         print(i)
stime= time.time()
path_f=Astar_alg(tuple([8.5,6.5,4.5] ),tuple( [47.5,32.5,9.5]),True)
# path_f=Astar_alg(tuple([23.40000077, 21.00000037, 10.99999963] ),tuple( [1.80000009 ,1.80000009 ,2.20000009]),True)
waypoints=[]
for i in path_f:
    waypoints.append(list(i))
time_f= np.asarray(time.time()-stime)
pathlength=np.asarray(find_path_lenth(waypoints))
print(pathlength,time_f)

70.0 1.9649786949157715


In [13]:
save_as_csv('NaN','/home/gyk/data/mypaths_path_astar_bigmap.csv',3,waypoints)

True

In [16]:
df_out.to_csv('/home/gyk/data/prm/evaluation_astar_bigmap.csv')